In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import re

In [ ]:
driver = webdriver.Chrome()
url_conad = 'https://spesaonline.conad.it/home'
driver.get(url_conad)

#rifiuto i cookie
selettore_cookie = driver.find_elements(By.CSS_SELECTOR, '#onetrust-reject-all-handler')
selettore_cookie[0].click() 

#clicca sul tuo indirizzo
selettore_indirizzo = driver.find_element(By.CSS_SELECTOR, "label[for='googleInputEntrypageLine1']")
driver.execute_script("arguments[0].click();", selettore_indirizzo)
                      
input_box = driver.find_element(By.ID, "googleInputEntrypageLine1")
input_box.clear()  # Clear any existing value
input_box.send_keys("20145")
                      
#clicca su verifica 
verifica_button = driver.find_element(By.CSS_SELECTOR, "button.submitButton")
verifica_button.click()
time.sleep(2)                      
                      
seleziona_button= driver.find_element(By.CSS_SELECTOR, "#ordina-e-ritira > div > div > button")
seleziona_button.click()
time.sleep(2)                      
                      
conad_button= driver.find_element(By.CSS_SELECTOR, "#ordina-ritira-scelta-pdv > div.section-view-negozi > div > div.lista-negozi-section > div > ul > li:nth-child(1) > div")
conad_button.click()
time.sleep(1)                     
                      
negozio_button= driver.find_element(By.CSS_SELECTOR, "#modal-onboarding-wrapper > div.modal-content.uk-align-center.uk-grid-margin.uk-first-column > div.bottom-section.btn-conferma-pdv > button")
negozio_button.click()
time.sleep(3)                    
                      
tutti_prod_button = driver.find_element(By.CSS_SELECTOR,"a[onclick*='Header.loadMiniBanners']")
tutti_prod_button.click()
time.sleep(3)                      

back office

identifico nomi di selettori e sottocategorie

In [ ]:
html = driver.page_source #interazione beautiful soup e eselenium
soup = BeautifulSoup(html, 'html.parser')

#tutte le sottocategorie
subcategory_elements = soup.find('div', class_='megamenu-second-level').find_all('li')
#nome delle sottocategorie
#in all_subcat mi appaiono i nomi di tutti gli elementi nel secondo drop down menu dopo tutti i 
#prodotti. 
#per poterlo usare successivamente, devo togliere i selettori da questa lista
#tuttavia due selettori, (piatti pronti - surgelati e gelati) sono sia selettori che sottocat
#plan of attack: rimuovo solo questi due da all_subcat nelle posizioni che hanno da selettore
#elimino i selettori che ho già tolto dalla lista di selettori da togliere
#rimuovo tutti i selettori rimanenti da all_subcat e lo salvo in subcat
all_subcat = [category.a.get_text(strip=True) for category in subcategory_elements]

effettuo operazioni in modo da togliere le macrocategorie dalle sottocategorie (piatti pronti - surgelati e gelati)

In [ ]:
#remove piatti pronti e surgelati e gelati (selettori) from all_subcat
# Specify the positions to remove
positions = [75, 85]

# Sort the positions in descending order to avoid index shifting
positions.sort(reverse=True)

# Remove elements at the specified positions
for pos in positions:
    del all_subcat[pos]

In [ ]:
#tutte le categorie
selettori_elements = soup.find('div', class_='uk-animation-slide-left uk-first-column').find_all('li')
#nome delle categorie
selettori_names = [selettore.a.get_text(strip=True) for selettore in selettori_elements]


identifico i selettori da togliere: tutti tranne quelli che compaiono anche come sottocategorie.

In [ ]:
selettori_da_togliere=selettori_names.copy()
selettori_da_togliere.remove('Piatti pronti')
selettori_da_togliere.remove('Surgelati e gelati')

ora ottengo subcat, cioè tutte le sottocategorie che posso usare mentre itero

In [ ]:
subcat = [x for x in all_subcat if x not in selettori_da_togliere]


In [ ]:
#counting the number of sottocategorie per ogni selettore
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
megamenu_div = soup.find('div', class_='megamenu-second-level')
div_elements = megamenu_div.find_all('div', class_='uk-hidden')

numero_per_sottocat = []

for div in div_elements:
    li_elements = div.find_all('li')
    #faccio meno uno perché nelle sottocategorie, il primo non lo considero
    li_count = len(li_elements)-1
    numero_per_sottocat.append(li_count)

print(numero_per_sottocat)

[6, 10, 1, 6, 6, 7, 7, 10, 6, 6, 9, 11, 8, 9, 2, 4, 9, 6, 3]


In [ ]:
 #identifico i selettori 
selettori=driver.find_elements(By.CSS_SELECTOR, "li.cat")
selettori_len=len(selettori)

In [ ]:

def find_prices(soup):
    price_elements = soup.find_all(class_='product-price-kg')
    # Extract the prices from the elements
    for element in price_elements:
            if 'override-color-red' in element.get('class', []):
                price1=element.get_text(strip=True)
                price = float(price1.split(' ')[0].replace(',', '.'))
                #calcolo la percentuale di sconto
                discount_span = element.find_previous(class_='badge-text')
                #trasformo lo sconto in numero intero
                discount = int(discount_span.get_text(strip=True).rstrip('%').lstrip('-'))
                #calcolo il prezzo non scontato
                new_price = round(price/((100-discount)/100),2)
                #lo riconverto in stringa
                string_risultato="{:,.1f}".format(new_price)
                #lo rimetto nel formato iniziale
                risultato = price1.replace(price1.split(' ')[0], string_risultato, 1)
                prices.append(risultato)
            else:
                price = element.get_text(strip=True)
                prices.append(price)
    return prices


def find_products(soup):
    div_elements = soup.find_all('div', class_='no-t-decoration product-description uk-position-relative')

    # Iterate over the <div> elements
    for div_element in div_elements:
        # Find the <h3> element within the current <div> element
        h3_element = div_element.find('h3')
        product_name = h3_element.text.strip()
        products.append(product_name)
        # Print or process the product name
    return products

def find_single_prices(soup):
    price_single=soup.find_all('div',class_="product-price")
    price_single
    for p in price_single:
        price_single_final= p.get_text(strip=True)
        prices_single.append(price_single_final)
    return prices_single

In [ ]:
data = pd.DataFrame({'products': [], 'prices_per_unitmeasurement': [],'unit_price': [], 'category': '','subcategory':''})

#ipotesi
total_prod=[]
count=[]


for i in (range(selettori_len-3)):
    driver.get(url_conad)
    time.sleep(0.5)
    tutti_prod_button = driver.find_element(By.CSS_SELECTOR,"a[onclick*='Header.loadMiniBanners']")
    tutti_prod_button.click()
    time.sleep(0.2)
    selettori=driver.find_elements(By.CSS_SELECTOR, "li.cat")
    time.sleep(0.2)
    selettori[i].click()
    category_links = driver.find_elements(By.CSS_SELECTOR, "[onclick^='Header.trackSecondLevel']")
    #vado sulla prima sottocategoria di un selettore nel drop down menu 'tutti prodotti'
    #i category links (che sarebbero le sottocategorie) dei selettori partono da 0 solo 
    #nel primo selettore e vanno fino a 145. il primo che ripete il nome del selettore non viene
    #contato
    #sum(numero_per_sottocat[:i]) clicks on the first element of the ith category
    category_links[sum(numero_per_sottocat[:i])].click()
    time.sleep(0.2)
    #itero per un numero di volte pari al numero di sottocategorie per ogni selettore
    #remember: j starts from 0.
    for j in range(numero_per_sottocat[i]):
    #ora sono nella prima sottocategoria di un selettore
    #ottengo tutti i link delle sottocategorie 
                category_list = driver.find_elements(By.CSS_SELECTOR, "ul.category-list li a")
                category_selectors = [category.get_attribute("href") for category in category_list]
    #per ogni sottocategoria, clicco sul suo link
                driver.get(category_selectors[j])
    #svuoto prodotti e prices perchè li rinnovo per ogni sottocategoria
                products = []
                prices_single=[]
                prices=[]
                time.sleep(0.5)
    #scorro tutte le pagine di una sottocategoria
                while True:
                            try:
                                html = driver.page_source #interazione beautiful soup e eselenium
                                soup = BeautifulSoup(html, 'html.parser')
                                find_prices(soup)
                                find_products(soup)
                                find_single_prices(soup)
                                selettori_names[i]
                                next_element = driver.find_element(By.CSS_SELECTOR,"a[aria-label='Pagina Successiva']")
                                next_element.click()
                                time.sleep(2.5)
                            except NoSuchElementException:
                                # No more pages to view
                                break
        #qua ho finito di scorrere tutte le pagine di un selettore
        #conto il numero di item per questa sottocategoria
                results_element = soup.find('b', class_='results')
                conteggio= int(results_element.text.strip().split()[0])
        #salvo il conteggio in una lista
                count.append(conteggio)
        #salvo i prodotti di una sottocateogira in un df data_sottocat
                total_prod=total_prod + products
        #il motivo per cui sottocat è subcat[sum(numero_per_sottocat[:i])+j] è che in subcat ho
        #tutte le sottocategorie insieme. con sum(numero_per_sottocat[:i]) arrivo fino al primo
        #elemento del selettore i e aggiungo j, il numero di 
     #di sottocategoria a cui sono arrivata nel current selettore. ricordo che j comincia da 0
                data_sottocat = pd.DataFrame({'products': products, 'prices_per_unitmeasurement': prices,'unit_price': prices_single, 'category': selettori_names[i],'subcategory':subcat[sum(numero_per_sottocat[:i])+j]})
                data=data.append(data_sottocat)

In [ ]:
data

,products,prices_per_unitmeasurement,unit_price,category,subcategory
0,"Arance Valencia Italia cal. 7 1,5 kg Percorso ...","2,65 € / Kg","3,98€",Frutta e verdura,Frutta fresca
1,Pompelmi Rossi Cal. 4/5 Cat. I,"2,48 € / Kg","1,98€",Frutta e verdura,Frutta fresca
2,Limone Costa d'Amalfi I.G.P. Primofiore Italia...,"5,56 € / Kg","2,78€",Frutta e verdura,Frutta fresca
3,"Limoni Verdello Italia 0,500 kg Verso Natura C...","3,96 € / Kg","1,98€",Frutta e verdura,Frutta fresca
4,Percorso Qualità Limoni Italia Cal. 4 Cat. I R...,"3,96 € / Kg","1,98€",Frutta e verdura,Frutta fresca
...,...,...,...,...,...
0,Enervit Power Crunchy Choco Drops 40 g,"38,75 € / Kg","1,55€",Prodotti dietetici e integratori,Alimentazione sportiva
1,Enervit Protein 32% Bar with Vitamins Lemon Ca...,"35,21 € / Kg","1,69€",Prodotti dietetici e integratori,Alimentazione sportiva
2,Enervit Sport Power Crunchy Bar Gusto Cookie 40 g,"38,75 € / Kg","1,55€",Prodotti dietetici e integratori,Alimentazione sportiva
3,Enervit The Protein Deal Choco & Vanilla Dream...,"38,00 € / Kg","2,09€",Prodotti dietetici e integratori,Alimentazione sportiva


In [ ]:
sum(count)

8004

In [ ]:
data.to_csv('conad_final.csv', index=False)

In [ ]:
datab = pd.read_csv('conad_final.csv')

In [ ]:
datab

,products,prices_per_unitmeasurement,unit_price,category,subcategory
0,"Arance Valencia Italia cal. 7 1,5 kg Percorso ...","2,65 € / Kg","3,98€",Frutta e verdura,Frutta fresca
1,Pompelmi Rossi Cal. 4/5 Cat. I,"2,48 € / Kg","1,98€",Frutta e verdura,Frutta fresca
2,Limone Costa d'Amalfi I.G.P. Primofiore Italia...,"5,56 € / Kg","2,78€",Frutta e verdura,Frutta fresca
3,"Limoni Verdello Italia 0,500 kg Verso Natura C...","3,96 € / Kg","1,98€",Frutta e verdura,Frutta fresca
4,Percorso Qualità Limoni Italia Cal. 4 Cat. I R...,"3,96 € / Kg","1,98€",Frutta e verdura,Frutta fresca
...,...,...,...,...,...
8099,Enervit Power Crunchy Choco Drops 40 g,"38,75 € / Kg","1,55€",Prodotti dietetici e integratori,Alimentazione sportiva
8100,Enervit Protein 32% Bar with Vitamins Lemon Ca...,"35,21 € / Kg","1,69€",Prodotti dietetici e integratori,Alimentazione sportiva
8101,Enervit Sport Power Crunchy Bar Gusto Cookie 40 g,"38,75 € / Kg","1,55€",Prodotti dietetici e integratori,Alimentazione sportiva
8102,Enervit The Protein Deal Choco & Vanilla Dream...,"38,00 € / Kg","2,09€",Prodotti dietetici e integratori,Alimentazione sportiva
